<a href="https://colab.research.google.com/github/tjdrhks0808/Project-Road_Accident/blob/main/Daejeon_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--------------------
# 사고 유형별 특징 파악
--------------------

In [ ]:
from geoband.API import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

import pydeck as pdk
import shapefile
import os
import re
import matplotlib 
matplotlib.font_manager._rebuild()
import seaborn as sns
from matplotlib import font_manager, rc

In [ ]:
acc_hist = pd.read_csv('1.대전광역시_교통사고내역(2017~2019).csv')
acc_cnt = gpd.read_file('2.대전광역시_교통사고격자(2017~2019).geojson')
trlight_pas = gpd.read_file('3.대전광역시_신호등(보행등).geojson')
trlight_car = gpd.read_file('4.대전광역시_신호등(차량등).geojson')
safe_zone = gpd.read_file('5.대전광역시_안전지대.geojson')
crosswalk = gpd.read_file('6.대전광역시_횡단보도.geojson')
sign_speed = gpd.read_file('7.대전광역시_도로속도표시.geojson')
noparking_zone = gpd.read_file('8.대전광역시_정차금지지대.geojson')
sign_traffic = gpd.read_file('9.대전광역시_교통안전표지.geojson')
cctv = gpd.read_file('10.대전광역시_교통CCTV.geojson')
pop_dong = pd.read_csv('11.대전광역시_동별_인구현황(2017~2019).csv')
pop_all = gpd.read_file('12.대전광역시_인구정보(총인구).geojson')
pop_sen = gpd.read_file('13.대전광역시_인구정보(고령).geojson')
pop_prd = gpd.read_file('14.대전광역시_인구정보(생산가능).geojson')
pop_chd = gpd.read_file('15.대전광역시_인구정보(유소년).geojson')
weather = pd.read_csv('16.대전광역시_기상데이터(2017~2019).csv')
traffic_link = gpd.read_file('17.대전광역시_교통링크(2018).geojson')
traffic_node = gpd.read_file('18.대전광역시_교통노드(2018).geojson')
traffic_road = gpd.read_file('19.대전광역시_상세도로망(2018).geojson')
traffic_amt = pd.read_csv('20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')
traffic_freq = pd.read_csv('21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')
traffic_time = pd.read_csv('22.대전광역시_평일_일별_혼잡시간강도(2018).csv')
building = gpd.read_file('23.대전광역시_도로명주소(건물).geojson')
building_area = gpd.read_file('24.대전광역시_건물연면적_격자.geojson')
bound_law = gpd.read_file('25.대전광역시_법정경계(시군구).geojson')
bound_law2 = gpd.read_file('26.대전광역시_법정경계(읍면동).geojson')
bound_adm = gpd.read_file('27.대전광역시_행정경계(읍면동).geojson')
daejeon = gpd.read_file('28.대전광역시_연속지적도.geojson')

In [ ]:
acc_cnt

In [ ]:
acc_hist.사고유형.value_counts()

단순하게 차대차, 차대사람, 차량단독으로 구분지을 수도 있지만 차대사람 
유형중에서도  
차도통행중, 보도통행중같은 경우는 사고에 영향을 끼치는 
    안전시설물들의 유형이 다를 것이라  
    예상된다.  
    따라서 교통안전시설물과의 인과관계를 분석
    하기 위해서는 사고 유형을 세분화해서 분석을 진행하였다.

### **정규식을 활용한 유형분류**

#### 사고유형별로 구분

In [ ]:
acc_car=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_per=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_alone=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])

for i in range(len(acc_hist)):                #차대차 유형만 분류한 데이터프레임 생성
    m = re.search('^차대차',acc_hist.사고유형[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_car=acc_car.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist)):              #차대사람 유형만 분류한 데이터프레임 생성
    m = re.search('^차대사람',acc_hist.사고유형[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_per=acc_per.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)

        
for i in range(len(acc_hist)):            #차량단독 유형만 분류한 데이터프레임 생성
    m = re.search('^차량단독',acc_hist.사고유형[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_alone=acc_alone.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)

분류된 데이터셋

In [ ]:
acc_alone.head(3)

In [ ]:
acc_per.head(3)

또한 반대로 사고유형을 단순화 시키는 것도 가능  
ex) '차대차-측면추돌' -> '차대차'

In [ ]:
acc_hist_simp=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])


for i in range(len(acc_hist)):                
    m = re.search('^차대차',acc_hist.사고유형[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        ls[2] = '차대차'
        acc_hist_simp=acc_hist_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist)):               
    m = re.search('^차대사람',acc_hist.사고유형[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        ls[2] = '차대사람'
        acc_hist_simp=acc_hist_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)

        
for i in range(len(acc_hist)):               
    m = re.search('^차량단독',acc_hist.사고유형[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        ls[2] = '차량단독'
        acc_hist_simp=acc_hist_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)

In [ ]:
acc_hist_simp

위와 같은 방식으로 운전자 성별, 연령대에 따른 분류도 가능함

#### 사고내용(정도)에 따른 분류

In [ ]:
acc_hist.사고내용.value_counts()

In [ ]:
acc_minor=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_fatal=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_dead=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_injury=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])

for i in range(len(acc_hist)):                #경상 유형만 분류한 데이터프레임 생성
    m = re.search('^경상사고',acc_hist.사고내용[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_minor=acc_minor.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist)):                #중상 유형만 분류한 데이터프레임 생성
    m = re.search('^중상사고',acc_hist.사고내용[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_fatal=acc_fatal.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist)):                #부상 유형만 분류한 데이터프레임 생성
    m = re.search('^부상신고사고',acc_hist.사고내용[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_injury=acc_injury.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist)):                #사망 유형만 분류한 데이터프레임 생성
    m = re.search('^사망사고',acc_hist.사고내용[i])
    if m:
        ls = list(acc_hist.iloc[i,:])
        acc_dead=acc_dead.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)

분류된 데이터셋

In [ ]:
acc_dead.head(3)

In [ ]:
acc_minor.head(3)

### 사고유형별 사고정도 비율 확인

In [ ]:
acc_car_cnt = len(acc_car)
acc_per_cnt = len(acc_per)
acc_alone_cnt = len(acc_alone)

In [ ]:
print(acc_car_cnt / (acc_car_cnt + acc_per_cnt + acc_alone_cnt))
print(acc_per_cnt / (acc_car_cnt + acc_per_cnt + acc_alone_cnt))
print(acc_alone_cnt / (acc_car_cnt + acc_per_cnt + acc_alone_cnt))

차대차 사고: 약 77%  
차대사람 사고: 약 20%  
차량단독 사고: 약 3%  
  
차대차, 차대사람 사고에 중점을 둬서 분석해야함

#### 차대차 사고에서의 사고정도 확인

In [ ]:
acc_car.사고내용.value_counts()

In [ ]:
print(14244 / (14244+3519+406+83))
print(3519 / (14244+3519+406+83))
print(406 / (14244+3519+406+83))
print(83 / (14244+3519+406+83))

#### 차대사람 사고에서의 사고정도 확인

In [ ]:
acc_per.사고내용.value_counts()

In [ ]:
print(2648 / (2648+1749+162+127))
print(1749 / (2648+1749+162+127))
print(162 / (2648+1749+162+127))
print(127 / (2648+1749+162+127))

경상사고: 약 56%  
중상사고: 약 37%  
부상신고사고: 약 3%  
사망사고: 약 2%

#### 차량단독 사고에서의 사고정도 확인

In [ ]:
acc_alone.사고내용.value_counts()

In [ ]:
print(373 / (373+250+49+40))
print(250 / (373+250+49+40))
print(49 / (373+250+49+40))
print(40 / (373+250+49+40))

경상사고: 약 52%  
중상사고: 약 35%  
부상신고사고: 약 6%  
사망사고: 약 5%

종합하면 중상사고와 사망사고를 더 위험하다고 했을때,  
사고가 났을 때 가장 위험도가 높은 순서대로 나열하면  
*차량단독사고(57%) > 차대사람(39%) > 차대차(19.4%)*이다.

In [ ]:
plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

ax[0].hist(acc_car.사고내용)              # 중상사고, 경상사고, 부상신고사고, 사망사고
ax[0].set_title("acc_car")

ax[1].hist(acc_per.사고내용)
ax[1].set_title("acc_person")

ax[2].hist(acc_alone.사고내용)
ax[2].set_title("acc_alone")


plt.show()

### 사고정도별 사고유형 비율 확인

이번에는 반대로 사고정도별로 각 사고유형이 얼마나 높은 비중을 차지하는지 알아보기로 했다

우선 사고유형을 단순화 하기로함

In [ ]:
acc_minor_simp=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_fatal_simp=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_dead_simp=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])
acc_injury_simp=pd.DataFrame(columns=['사고일','시군구','사고유형','법규위반','사고내용',
                            '사망자수','중상자수','경상자수','부상신고자수','가해운전자차종',
                            '가해운전자연령대','가해운전자성별','피해자운전자차종','피해운전자연령대',
                            '피해운전자성별','gid'])

for i in range(len(acc_hist_simp)):                #경상 유형만 분류한 데이터프레임 생성
    m = re.search('^경상사고',acc_hist_simp.사고내용[i])
    if m:
        ls = list(acc_hist_simp.iloc[i,:])
        acc_minor_simp=acc_minor_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist_simp)):                #중상 유형만 분류한 데이터프레임 생성
    m = re.search('^중상사고',acc_hist_simp.사고내용[i])
    if m:
        ls = list(acc_hist_simp.iloc[i,:])
        acc_fatal_simp=acc_fatal_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist_simp)):                #부상 유형만 분류한 데이터프레임 생성
    m = re.search('^부상신고사고',acc_hist_simp.사고내용[i])
    if m:
        ls = list(acc_hist_simp.iloc[i,:])
        acc_injury_simp=acc_injury_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)
        
for i in range(len(acc_hist_simp)):                #사망 유형만 분류한 데이터프레임 생성
    m = re.search('^사망사고',acc_hist_simp.사고내용[i])
    if m:
        ls = list(acc_hist_simp.iloc[i,:])
        acc_dead_simp=acc_dead_simp.append({'사고일':ls[0],'시군구':ls[1],'사고유형':ls[2],'법규위반':ls[3],
                               '사고내용':ls[4],'사망자수':ls[5],'중상자수':ls[6],'경상자수':ls[7],
                               '부상신고자수':ls[8],'가해운전자차종':ls[9],'가해운전자연령대':ls[10],
                                '가해운전자성별':ls[11],
                               '피해자운전자차종':ls[12],'피해운전자연령대':ls[13],'피해운전자성별':ls[14],
                                'gid':ls[15]},ignore_index=True)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 5))

ax[0].hist(acc_dead_simp.사고유형)              #왼쪽부터 차대차, 차대사람, 차량단독
ax[0].set_title("acc_dead")

ax[1].hist(acc_fatal_simp.사고유형)
ax[1].set_title("acc_fatal")

ax[2].hist(acc_minor_simp.사고유형)
ax[2].set_title("acc_minor")

ax[3].hist(acc_injury_simp.사고유형)
ax[3].set_title("acc_injury")

plt.rcParams['font.family'] = 'NanumGothic'
plt.show()

사망자가 발생한 사고의 경우는 차대사람 사고 유형의 비율이 가장 높았으나, 나머지 중상,경상,부상은 차대차 사고 유형의 비율이 높음을 알 수 있다.

**결론종합**  
  
**1. 사고가 발생했을 때 가장 위험도가 높은 사고 유형은 차량단독사고 유형이고 그 다음은 차대사람 유형이다**  
**2. 실제로 사망자 발생 사고에서 가장 높은 비중을 차지하는 유형은 차대사람사고 유형이다**  

**위험도가 높은 사고는 차량단독이지만 실제로 사고발생건수에서 차지하는 비율이 적어 실제 사망자수와 중상자수에 많은 영향을 끼치는 사고들은 차대사람, 차대차사고 유형이다.**

**따라서 중상,사망 사고의 발생률을 낮추기 위해서 차량단독의 경우에는 상대적으로 발생건수를 줄이기 위한 노력보다 사고의 위험성을 감소시키는 장치가 필요하고, 차대차 사고의 경우는 발생 건수 자체를 줄이는 방법에 더 비중을 둬야한다. 차대사람은 위험성도 낮추고 사고 발생건수도 감소시켜야 한다.** 

--------------------
# 연령대별 특징 파악
--------------------

In [ ]:
from geoband.API import *

from tqdm import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
import geopy.distance

import pydeck as pdk
import shapely
import shapefile
import os
import re
import mapboxgl
from mapboxgl.viz import *
import seaborn as sns

import platform
import requests; from urllib.parse import urlparse

import warnings
warnings.filterwarnings(action='ignore') #경고 메시지 무시
from IPython.display import display #print가 아닌 display()로 연속 출력

In [ ]:
total = gpd.read_file('full_data.geojson') #정제전 통합 데이터
data = gpd.read_file('daejeon_data.geojson')

## 격자별 평균 연령 계산

In [ ]:
tmp = total[['gid','acci_cnt','attack_under_10 ','attack_10', 'attack_20',
       'attack_30', 'attack_40', 'attack_50', 'attack_60', 'attack_70',
       'attack_80', 'attack_90' ]] # 가해자
tmp2 = total[['gid','acci_cnt','victim_under_10','victim_10', 'victim_20',
       'victim_30', 'victim_40', 'victim_50', 'victim_60', 'victim_70',
       'victim_80', 'victim_90' ]] #피해자

In [ ]:
tmp['avg_age_a'] = tmp['attack_under_10 ']*5 + tmp['attack_10']*15 + tmp['attack_20']*25 + tmp['attack_30']*35 + tmp['attack_40']*45 + tmp['attack_50']*55 + tmp['attack_60']*65 + tmp['attack_70']*75 + tmp['attack_80']*85 + tmp['attack_90']*95
tmp['avg_age_a'] = tmp['avg_age_a'] / tmp['acci_cnt']

tmp2['avg_age_v'] = tmp2['victim_under_10']*5 + tmp2['victim_10']*15 + tmp2['victim_20']*25 + tmp2['victim_30']*35 + tmp2['victim_40']*45 + tmp2['victim_50']*55 + tmp2['victim_60']*65 + tmp2['victim_70']*75 + tmp2['victim_80']*85 + tmp2['victim_90']*95
tmp2['avg_age_v'] = tmp2['avg_age_v'] / tmp2['acci_cnt']

In [ ]:
eda = tmp.merge(tmp2[['gid', 'avg_age_v']], on='gid')

In [ ]:
sns.distplot(eda['avg_age_a'], 

             color="blue", label="avg_age_a", bins=10, kde=False)



sns.distplot(eda['avg_age_v'], 

             color="red", label="avg_age_v", bins=10, kde=False)


plt.xlabel('age')
plt.ylabel('count')

plt.legend(title="type")

plt.show()


3년간 교통사고 의 격자별 가해자(age_a)와 피해자(age_v) 평균 연령 분포

두 분포를 봤을 때, 평균 연령이 가해자냐 피해자냐에 따라 크게 변화하지는 않는 것으로 보인다. 다만, 피해자의 연령분포가 가해자에 비해 조금 더 어린 것으로 나타났다.

## 평균 연령과 사고 건수의 관계

In [ ]:
sns.set_style("darkgrid")
sns.jointplot(data=eda, x="avg_age_a", y="acci_cnt", color='grey',joint_kws={'color':'green'},marginal_kws=dict(bins=15))

In [ ]:
sns.jointplot(data=eda, x="avg_age_v", y="acci_cnt", color='grey',joint_kws={'color':'green'},marginal_kws=dict(bins=15))

사고건수와 가해자, 피해자의 평균 연령 분포는 선형관계가 보이지 않는다.

건수 대신 위험도를 넣고 다시 보자.

## 평균 연령과 사고 위험도 간의 관계

In [ ]:
eda = eda.merge(data[['gid', 'acci_risk']], on='gid')

In [ ]:
sns.jointplot(data=eda, x="avg_age_a", y="acci_risk", marginal_kws=dict(bins=15, color='grey'),color='green')

In [ ]:
sns.jointplot(data=eda, x="avg_age_v", y="acci_risk", marginal_kws=dict(bins=15, color='grey'),color='green')

건수 대신 위험도를 넣어도 연령대와 위험도간 선형 관계는 보이지 않는다.

사고 유형별로 나누어서 다시 관계를 살펴보자.

In [ ]:
eda= eda.merge(data[['gid', 'acci_risk_ctc', 'acci_risk_cth']], on='gid')

In [ ]:
sns.jointplot(data=eda, x="avg_age_a", y="acci_risk_ctc", marginal_kws=dict(bins=15, color='grey'),color='green')

In [ ]:
sns.jointplot(data=eda, x="avg_age_v", y="acci_risk_ctc", marginal_kws=dict(bins=15, color='grey'),color='green')

In [ ]:
sns.jointplot(data=eda, x="avg_age_a", y="acci_risk_cth", marginal_kws=dict(bins=15, color='grey'),color='green')

In [ ]:
sns.jointplot(data=eda, x="avg_age_v", y="acci_risk_cth", marginal_kws=dict(bins=15, color='grey'),color='green')

위험도를 사고 유형별로 분리해서 봐도 나이와 상관관계(선형성)을 보이지 않았다.

**결론: 특정 나이대가 특정 유형의 사고에 집중되어 있거나 가해자나 피해자의 연령대가 쏠려있다던지 하는 경향성이 전혀 보이지 않는다. 30-40대의 사고건수나 위험도가 높은 것은 인구 분포의 특성상 그 나이대가 가장 많은 나이대이기 때문인 것으로 보인다.**

**따라서 사고 관련자의 나이대는 확률적 요소에 의해 결정 되는 것으로 생각된다.**

--------------------
# 주요 변수 시각화
--------------------

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
from geoband.API import *
import mapboxgl
from mapboxgl.viz import *
from mapboxgl.utils import create_color_stops

from mapboxgl_notebook.map import MapboxMap
from mapboxgl_notebook.sources import GeoJSONSource
from mapboxgl_notebook.layers import PointCircleLayer, LineStringLineLayer, PolygonFillLayer
from mapboxgl_notebook.properties import Paint
from mapboxgl_notebook.interactions import ClickInteraction, HoverInteraction
from mapboxgl.utils import *

In [ ]:
data = gpd.read_file('daejeon_data.geojson')

In [ ]:
danger = 'daejeon_data.geojson'

import json
with open(danger) as f:
    danger = json.loads(f.read())

In [ ]:
from mapboxgl.utils import create_color_stops
token = 'pk.eyJ1Ijoia2todiIsImEiOiJja210ZDlnaGIwcXA2MndwZngwc2Vwd3pqIn0.r0aLC7SrHVBuYiA1kyBVXQ'

In [ ]:
center = [127.39450577700798, 36.33982630520595]

In [ ]:
from mapboxgl.utils import (
    create_numeric_stops,
    create_color_stops
)

## 대전시 교통사고 건수 분포

In [ ]:
viz = HeatmapViz(
    danger,
    access_token=token,
    weight_property="acci_cnt",
    weight_stops=create_numeric_stops([10,20,30,40,50], 0, 1),
    color_stops=create_color_stops([0.1, 0.3, 0.5, 0.7, 0.9], colors='YlOrRd'),
    radius_stops=create_numeric_stops([8, 10, 12, 14, 16], 3, 12),
    center=center,
    zoom=10
)
viz.show()

도로를 따라서 높은 밀도로 사고 발생, 유성구, 서구(계룡료)에 집중적으로 사고가 발생하는 모양이다.

## 교통량 분포

In [ ]:
viz = HeatmapViz(
    danger,
    access_token=token,
    weight_property="traffic_total_sum_inter",
    weight_stops=create_numeric_stops([100000,150000,200000,250000,300000,350000,450000,500000], 0, 1),
    color_stops=create_color_stops([0.1, 0.3, 0.5, 0.7, 0.9], colors='RdPu'),
    radius_stops=create_numeric_stops([8, 10, 12, 14, 16], 3, 12),
    center=center,
    zoom=10
)
viz.show()

대전을 가로지르는 중심 도로를 따라서 높은 교통량이 분포

## 교통량+사고 분포

In [ ]:
color_breaks3 = list(range(0,500000, 100000))
color_stops3 =create_color_stops(color_breaks3, colors='RdPu') # color stops setting은 다르게 해줘야 눈에 보임

viz4 = ChoroplethViz(access_token=token
                   ,data=danger,
                   color_property='traffic_total_sum_inter',
                   color_stops=color_stops3,
                   center=center,
                   zoom=10)                      

viz4.bearing = -15 # 왼쪽으로 돌리고~
viz4.pitch = 45 #오른쪽으로 돌리고~

viz4.height_property = 'acci_cnt'

numeric_stops = create_numeric_stops([10,20,30,40,50],0,1000)

viz4.height_stops = numeric_stops
viz4.height_function_type = 'interpolate'

viz4.show()

교통량이 많은 곳에 대체적으로 사고 건수가 많은 편이다.(사고건수: 높이) 그러나 교통량이 많아도 사고 건수가 많지 않은 곳도 존재한다.

## 교통량+사고 위험도

In [ ]:
color_breaks3 = list(range(0,500000, 100000))
color_stops3 =create_color_stops(color_breaks3, colors='Reds') # color stops setting은 다르게 해줘야 눈에 보임

viz4 = ChoroplethViz(access_token=token
                   ,data=danger,
                   color_property='traffic_total_sum_inter',
                   color_stops=color_stops3,
                   center=center,
                   zoom=10)                      

viz4.bearing = -15 # 왼쪽으로 돌리고~
viz4.pitch = 45 #오른쪽으로 돌리고~

viz4.height_property = 'acci_risk'

numeric_stops = create_numeric_stops([10,20,30,40,50],0,1000)

viz4.height_stops = numeric_stops
viz4.height_function_type = 'interpolate'

viz4.show()

교통량이 많은 곳이 사고 위험도도 높다고 나타났다. 사고건수보다 좀 더 뚜렷한 관계가 보인다.

## 사고유형별 주요 변수와의 상관관계 및 시각화

### 신호등(보행등)과의 관계

In [ ]:
trt_pas = trlight_pas.copy()

trt_pas['cnt'] = list(np.repeat(1,7407))

trt_pas = trt_pas.drop(['gu','dong','jibun','loc_cd','sgnl_drn_cd','sgnl_knd_cd'], axis=1) 
#위치만 알아보기 위해 나머지 정보값들을 배재하고 시각화하기로함

In [ ]:
acc_cnt_1 = acc_cnt.copy()
acc_per_1 = acc_per.copy()

acc_per_1 = acc_per_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_per_1 = acc_per_1.rename(columns={'사고일':'차대사람_사고횟수'})

join_per = pd.merge(acc_cnt_1, acc_per_1, on='gid', how='left')
join_per = join_per.fillna(0)
join_per.차대사람_사고횟수 = join_per.차대사람_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_per = join_per.drop(['acci_cnt'],axis=1) 

In [ ]:
data1 = gpd.sjoin(join_per,trt_pas,how='left', op='intersects')

data1.cnt = data1.cnt.fillna(0)

data_1 = data1.groupby('gid',as_index=False).sum()

In [ ]:
data_1.corr()

차대사람 사고건수와 신호등(보행등, 이하 보행등)의 상관관계는 0.4로 상관관계를 보인다

다음은 차대차

In [ ]:
acc_car_1 = acc_car.copy()

acc_car_1 = acc_car_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_car_1 = acc_car_1.rename(columns={'사고일':'차대차_사고횟수'})

join_car = pd.merge(acc_cnt_1, acc_car_1, on='gid', how='left')
join_car = join_car.fillna(0)
join_car.차대차_사고횟수 = join_car.차대차_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_car = join_car.drop(['acci_cnt'],axis=1) 

In [ ]:
data2 = gpd.sjoin(join_car,trt_pas,how='left', op='intersects')
data2.cnt = data2.cnt.fillna(0)
data_2 = data2.groupby('gid',as_index=False).sum()

data_2.corr()

특이하게도 차대차 사고건수와 보행등의 상관계수가 차대사람사고-보행등의 상관계수보다 높게나온다.

이번엔 차량단독

In [ ]:
acc_alone_1 = acc_alone.copy()

acc_alone_1 = acc_alone_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_alone_1 = acc_alone_1.rename(columns={'사고일':'차량단독_사고횟수'})

join_alone = pd.merge(acc_cnt_1, acc_alone_1, on='gid', how='left')
join_alone = join_alone.fillna(0)
join_alone.차량단독_사고횟수 = join_alone.차량단독_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_alone = join_alone.drop(['acci_cnt'],axis=1) 

data3 = gpd.sjoin(join_alone,trt_pas,how='left', op='intersects')
data3.cnt = data3.cnt.fillna(0)
data_3 = data3.groupby('gid',as_index=False).sum()

data_3.corr()

정리하자면 차량단독, 차대차, 차대사람과 보행등의 개수의 상관계수는 *0.35, 0.6, 0.41*이다.우선 보행등과 교통사고간에 긍정적인 영향 관계가 있다는 것을 알 수 있다.특히 차대차와 보행등의 상관관계가 높다.

### 신호등(차량등)과의 상관관계

In [ ]:
trt_car = trlight_car.copy()

trt_car['cnt'] = list(np.repeat(1,len(trt_car)))

trt_car = trt_car.drop(['gu','dong','jibun','loc_cd','sgnl_drn_cd','sgnl_knd_cd'], axis=1) 

차대사람 vs 차량등

In [ ]:
acc_per_1 = acc_per.copy()

acc_per_1 = acc_per_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_per_1 = acc_per_1.rename(columns={'사고일':'차대사람_사고횟수'})

join_per = pd.merge(acc_cnt_1, acc_per_1, on='gid', how='left')
join_per = join_per.fillna(0)
join_per.차대사람_사고횟수 = join_per.차대사람_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_per = join_per.drop(['acci_cnt'],axis=1) 

data1 = gpd.sjoin(join_per,trt_car,how='left', op='intersects')
data1.cnt = data1.cnt.fillna(0)
data_1 = data1.groupby('gid',as_index=False).sum()

data_1.corr()

차대차 vs 차량등

In [ ]:
acc_car_1 = acc_car.copy()

acc_car_1 = acc_car_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_car_1 = acc_car_1.rename(columns={'사고일':'차대차_사고횟수'})

join_car = pd.merge(acc_cnt_1, acc_car_1, on='gid', how='left')
join_car = join_car.fillna(0)
join_car.차대차_사고횟수 = join_car.차대차_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_car = join_car.drop(['acci_cnt'],axis=1) 

data2 = gpd.sjoin(join_car,trt_car,how='left', op='intersects')
data2.cnt = data2.cnt.fillna(0)
data_2 = data2.groupby('gid',as_index=False).sum()

data_2.corr()

차량단독 vs 차량등

In [ ]:
acc_alone_1 = acc_alone.copy()

acc_alone_1 = acc_alone_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_alone_1 = acc_alone_1.rename(columns={'사고일':'차량단독_사고횟수'})

join_alone = pd.merge(acc_cnt_1, acc_alone_1, on='gid', how='left')
join_alone = join_alone.fillna(0)
join_alone.차량단독_사고횟수 = join_alone.차량단독_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_alone = join_alone.drop(['acci_cnt'],axis=1) 

data3 = gpd.sjoin(join_alone,trt_car,how='left', op='intersects')
data3.cnt = data3.cnt.fillna(0)
data_3 = data3.groupby('gid',as_index=False).sum()

data_3.corr()

정리하자면 차량단독, 차대차, 차대사람과 차량등의 개수의 상관계수는 *0.35, 0.57, 0.48*이다. 앞선 교통사고 vs 보행등과 비슷한 양상을 보인다.  
이를 통해 신호등과 교통사고는 양의 상관관계를 지니고 있음을 알 수 있다.

### __시각화__

In [ ]:
acc_per_1 = acc_per.copy()

acc_per_1 = acc_per_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_per_1 = acc_per_1.rename(columns={'사고일':'차대사람_사고횟수'})

acc_cnt_1 = acc_cnt.copy()

join_per = pd.merge(acc_cnt_1, acc_per_1, on='gid', how='left')
join_per = join_per.fillna(0)
join_per.차대사람_사고횟수 = join_per.차대사람_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_per = join_per.drop(['acci_cnt'],axis=1)    

In [ ]:
join_per = join_per[join_per.차대사람_사고횟수!=0]

# 시각화를 위해 사고횟수값이 0인 행들 제거

join_per.to_file('join_per.geojson', driver='GeoJSON')

join_per = 'join_per.geojson'

import json
with open(join_per) as f:
    join_per = json.loads(f.read())

In [ ]:
trt_pas = trlight_pas.copy()

trt_pas['cnt'] = list(np.repeat(1,7407))

trt_pas = trt_pas.drop(['gu','dong','jibun','loc_cd','sgnl_drn_cd','sgnl_knd_cd'], axis=1) 
#위치만 알아보기 위해 나머지 정보값들을 배재하고 시각화하기로함

In [ ]:
trt_pas.to_file('trt_pas.geojson', driver='GeoJSON')

trt_pas = 'trt_pas.geojson'

import json
with open(trt_pas) as f:
    trt_pas = json.loads(f.read())

#### 보행등과 차대사람 사고 유형 오버레이어

In [ ]:
source1 = GeoJSONSource(join_per, source_id='차대사람_사고횟수')    #멀티폴리곤 레이어

paint =Paint(
    fill_color = 'rgba(255,246,143,0.4)'
    #fill_outline_color = 'rgba()'   테두리 채우는 옵션
            )
layer1 = PolygonFillLayer(source1, paint = paint)
interaction = ClickInteraction(layer1)


source2 = GeoJSONSource(trt_pas, source_id ='cnt')           #포인트 레이어

paint2 = Paint(circle_radius=2, circle_color = '#a11515')
layer2 = PointCircleLayer(source2,paint=paint2)



mapbox_map = MapboxMap(
                center = [127.378953,36.321655],
                zoom = 10.7,
                access_token = token,
                sources =[source1,source2],
                layers=[layer1,layer2],
                interactions=[interaction],
                style='mapbox://styles/mapbox/navigation-guidance-night-v4'
)
mapbox_map.show()

그림만 봐도 강한 상관관계가 있는 것을 직관적으로 알 수 있다

#### 보행등과 차대차 사고 유형 오버레이어

In [ ]:
acc_car_1 = acc_car.copy()

acc_car_1 = acc_car_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_car_1 = acc_car_1.rename(columns={'사고일':'차대차_사고횟수'})

acc_cnt_1 = acc_cnt.copy()

join_car = pd.merge(acc_cnt_1, acc_car_1, on='gid', how='left')
join_car = join_car.fillna(0)
join_car.차대차_사고횟수 = join_car.차대차_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_car = join_car.drop(['acci_cnt'],axis=1)

In [ ]:
join_car = join_car[join_car.차대차_사고횟수!=0]

# 시각화를 위해 사고횟수값이 0인 행들 제거

join_car.to_file('join_car.geojson', driver='GeoJSON')

join_car = 'join_car.geojson'

import json
with open(join_car) as f:
    join_car = json.loads(f.read())

In [ ]:
source1 = GeoJSONSource(join_car, source_id='차대차_사고횟수')    #멀티폴리곤 레이어

paint =Paint(
    fill_color = 'rgba(219,197,254,0.5)'
    #fill_outline_color = 'rgba()'   테두리 채우는 옵션
            )
layer1 = PolygonFillLayer(source1, paint = paint)
interaction = ClickInteraction(layer1)


source2 = GeoJSONSource(trt_pas, source_id ='cnt')           #포인트 레이어

paint2 = Paint(circle_radius=2, circle_color = '#a11515')
layer2 = PointCircleLayer(source2,paint=paint2)



mapbox_map = MapboxMap(
                center = [127.378953,36.321655],
                zoom = 10.7,
                access_token = token,
                sources =[source1,source2],
                layers=[layer1,layer2],
                interactions=[interaction],
                style='mapbox://styles/mapbox/navigation-guidance-night-v4'
)
mapbox_map.show()

이 그림 역시 강한 상관관계가 있는 것을 직관적으로 알 수 있다

#### 보행등과 차량단독 사고 유형 오버레이어

In [ ]:
acc_alone_1 = acc_alone.copy()

acc_alone_1 = acc_alone_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_alone_1 = acc_alone_1.rename(columns={'사고일':'차량단독_사고횟수'})

acc_cnt_1 = acc_cnt.copy()

join_alone = pd.merge(acc_cnt_1, acc_alone_1, on='gid', how='left')
join_alone = join_alone.fillna(0)
join_alone.차량단독_사고횟수 = join_alone.차량단독_사고횟수.astype('int')                  #사고횟수 데이터와 차대사람 사고횟수를 조인

join_alone = join_alone.drop(['acci_cnt'],axis=1)   

In [ ]:
join_alone = join_alone[join_alone.차량단독_사고횟수!=0]

# 시각화를 위해 사고횟수값이 0인 행들 제거

join_alone.to_file('join_alone.geojson', driver='GeoJSON')

join_alone = 'join_alone.geojson'

import json
with open(join_alone) as f:
    join_alone = json.loads(f.read())

In [ ]:
source1 = GeoJSONSource(join_alone, source_id='차량단독_사고횟수')    #멀티폴리곤 레이어

paint =Paint(
    fill_color = 'rgba(204,254,197,0.9)'
    #fill_outline_color = 'rgba()'   테두리 채우는 옵션
            )
layer1 = PolygonFillLayer(source1, paint = paint)
interaction = ClickInteraction(layer1)


source2 = GeoJSONSource(trt_pas, source_id ='cnt')           #포인트 레이어

paint2 = Paint(circle_radius=2, circle_color = '#a11515')
layer2 = PointCircleLayer(source2,paint=paint2)



mapbox_map = MapboxMap(
                center = [127.378953,36.321655],
                zoom = 10.7,
                access_token = token,
                sources =[source1,source2],
                layers=[layer1,layer2],
                interactions=[interaction],
                style='mapbox://styles/mapbox/navigation-guidance-night-v4'
)
mapbox_map.show()

#### *차량등*과 차대차 사고 유형 오버레이어

In [ ]:
trt_car.to_file('trt_car.geojson', driver='GeoJSON')

trt_car = 'trt_car.geojson'

import json
with open(trt_car) as f:
    trt_car = json.loads(f.read())

In [ ]:
source1 = GeoJSONSource(join_car, source_id='차대차_사고횟수')    #멀티폴리곤 레이어

paint =Paint(
    fill_color = 'rgba(204,254,197,0.9)'
    #fill_outline_color = 'rgba()'   테두리 채우는 옵션
            )
layer1 = PolygonFillLayer(source1, paint = paint)
interaction = ClickInteraction(layer1)


source2 = GeoJSONSource(trt_car, source_id ='cnt')           #포인트 레이어

paint2 = Paint(circle_radius=2, circle_color = '#a11515')
layer2 = PointCircleLayer(source2,paint=paint2)



mapbox_map = MapboxMap(
                center = [127.378953,36.321655],
                zoom = 10.7,
                access_token = token,
                sources =[source1,source2],
                layers=[layer1,layer2],
                interactions=[interaction],
                style='mapbox://styles/mapbox/navigation-guidance-night-v4'
)
mapbox_map.show()

## 안전지대와의 상관관계

이번엔 사고유형별로 안전지대와의 상관성을 알아보기로 했다

**안전지대**

In [ ]:
sf = safe_zone.copy()

sf['cnt'] = list(np.repeat(1,len(sf)))
sf = sf.drop(['gu','dong','jibun','loc_cd','len'],axis=1)  
#geometry값 빼고 날려줌

In [ ]:
sf.to_file('sf.geojson', driver='GeoJSON')

sf = 'sf.geojson'

import json
with open(sf) as f:
    sf = json.loads(f.read())

In [ ]:
match_color_stops = [[1, 'rgb(241,219,66)']]



viz = ChoroplethViz(access_token=token
                    ,data=sf,
                   color_property='cnt',
                   color_stops=match_color_stops,
                   color_function_type='match',
                   center=center,
                   zoom=10.7,
                   style='mapbox://styles/mapbox/navigation-guidance-night-v4'
                   )
# data의 property값을 확인하여 속성을 지정

viz.show()

안전지대가 사고 발생에 영향을 끼치는지 알아보자.

In [ ]:
#우선 차대사람사고가 발생한 지역의 차대사람사고횟수와 안전지대의 개수를 조인
acc_per_1 = acc_per.copy()                                                         #차대사람 사고유형지역의 차대사람 사고횟수

acc_per_1 = acc_per_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_per_1 = acc_per_1.rename(columns={'사고일':'차대사람_사고횟수'})

join_per = pd.merge(acc_cnt_1, acc_per_1, on='gid', how='left')
join_per = join_per.fillna(0)                 
join_per = join_per.drop(['acci_cnt'],axis=1) 

In [ ]:
sf = safe_zone.copy()

sf['cnt'] = list(np.repeat(1,len(sf)))
sf = sf.drop(['gu','dong','jibun','loc_cd','len'],axis=1)  

In [ ]:
sf_per = gpd.sjoin(join_per, sf,op='intersects')

In [ ]:
sf_per_cnt = sf_per.groupby('gid',as_index=False).count().iloc[:,0:2]
sf_per_cnt = sf_per_cnt.rename(columns={'geometry':'safe_cnt'})

In [ ]:
sf_per = sf_per.drop(['index_right'],axis=1)

sf_per = sf_per.drop_duplicates()

In [ ]:
sf_per_new = pd.merge(sf_per_cnt,sf_per,on='gid',how='inner')

In [ ]:
sf_per_new = sf_per_new.drop(['cnt'],axis=1)  

In [ ]:
sf_per_geo = sf_per_new.drop(['geometry'],axis=1)    

In [ ]:
sf_per_geo = pd.merge(acc_cnt_1,sf_per_geo,on='gid',how='left')
sf_per_geo = sf_per_geo.fillna(0)
sf_per_geo = sf_per_geo.drop(['acci_cnt'],axis=1) 

In [ ]:
sf_per_geo.corr()

안전지대와 차대사람 사고건수의 상관관계는 0.33이다.

#### 차대차 vs 안전지대

In [ ]:
acc_car_1 = acc_car.copy()                                                         #차대차 사고유형지역의 차대사람 사고횟수

acc_car_1 = acc_car_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_car_1 = acc_car_1.rename(columns={'사고일':'차대차_사고횟수'})

join_car = pd.merge(acc_cnt_1, acc_car_1, on='gid', how='left')
join_car = join_car.fillna(0)                 
join_car = join_car.drop(['acci_cnt'],axis=1) 

In [ ]:
sf_car = gpd.sjoin(join_car, sf,op='intersects')

sf_car_cnt = sf_car.groupby('gid',as_index=False).count().iloc[:,0:2]
sf_car_cnt = sf_car_cnt.rename(columns={'geometry':'safe_cnt'})

In [ ]:
sf_car = sf_car.drop(['index_right'],axis=1)
sf_car = sf_car.drop_duplicates()

sf_car_new = pd.merge(sf_car_cnt,sf_car,on='gid',how='inner')
sf_car_new = sf_car_new.drop(['cnt'],axis=1)  

In [ ]:
sf_car_geo = sf_car_new.drop(['geometry'],axis=1)        #다시 전체 acc_cnt데이터와 결합하기 위해 지오메트리 드랍

sf_car_geo = pd.merge(acc_cnt_1,sf_car_geo,on='gid',how='left')
sf_car_geo = sf_car_geo.fillna(0)
sf_car_geo = sf_car_geo.drop(['acci_cnt'],axis=1) 

sf_car_geo.corr()

차대차사고건수와 안전지대의 상관계수는 0.49

#### 차량단독vs안전지대

In [ ]:
acc_alone_1 = acc_alone.copy()                                                         #차대차 사고유형지역의 차대사람 사고횟수

acc_alone_1 = acc_alone_1.groupby('gid', as_index=False).count().iloc[:, 0:2]
acc_alone_1 = acc_alone_1.rename(columns={'사고일':'차량단독_사고횟수'})

join_alone = pd.merge(acc_cnt_1, acc_alone_1, on='gid', how='left')
join_alone = join_alone.fillna(0)                 
join_alone = join_alone.drop(['acci_cnt'],axis=1) 

In [ ]:
sf_alone = gpd.sjoin(join_alone, sf,op='intersects')

sf_alone_cnt = sf_alone.groupby('gid',as_index=False).count().iloc[:,0:2]
sf_alone_cnt = sf_alone_cnt.rename(columns={'geometry':'safe_cnt'})

In [ ]:
sf_alone = sf_alone.drop(['index_right'],axis=1)
sf_alone = sf_alone.drop_duplicates()

sf_alone_new = pd.merge(sf_alone_cnt,sf_alone,on='gid',how='inner')
sf_alone_new = sf_alone_new.drop(['cnt'],axis=1)  

In [ ]:
sf_alone_geo = sf_alone_new.drop(['geometry'],axis=1)        #다시 전체 acc_cnt데이터와 결합하기 위해 지오메트리 드랍

sf_alone_geo = pd.merge(acc_cnt_1,sf_alone_geo,on='gid',how='left')
sf_alone_geo = sf_alone_geo.fillna(0)
sf_alone_geo = sf_alone_geo.drop(['acci_cnt'],axis=1) 

sf_alone_geo.corr()

정리하면 차량단독, 차대차, 차대사람 사고와 안전지대의 개수의 상관계수는 *0.25, 0.49, 0.33*임  
앞과 비슷한 양상을 나타낸다.

## 차대차 + 안전지대 오버레이어

In [ ]:
join_car = join_car[join_car.차대차_사고횟수!=0]

join_car.to_file('join_car.geojson', driver='GeoJSON')

join_car = 'join_car.geojson'

import json
with open(join_car) as f:
    join_car = json.loads(f.read())

In [ ]:
sf.to_file('sf_car.geojson', driver='GeoJSON')

sf = 'sf.geojson'

import json
with open(sf) as f:
    sf = json.loads(f.read())

In [ ]:
source1 = GeoJSONSource(join_car, source_id='차대차_사고횟수')    #멀티폴리곤 레이어

paint =Paint(
    fill_color = 'rgba(204,163,255,0.3)'
    #fill_outline_color = 'rgba()'   테두리 채우는 옵션
            )
layer1 = PolygonFillLayer(source1, paint = paint)
interaction = ClickInteraction(layer1)


source2 = GeoJSONSource(sf, source_id ='cnt')           #포인트 레이어
paint2 =Paint(
    fill_color = 'rgb(255,0,0)'
            )
layer2 = PolygonFillLayer(source2, paint = paint2)
interaction = ClickInteraction(layer2)




mapbox_map = MapboxMap(
                center = [127.378953,36.321655],
                zoom = 10.7,
                access_token = token,
                sources =[source1,source2],
                layers=[layer1,layer2],
                interactions=[interaction],
                style='mapbox://styles/mapbox/navigation-guidance-night-v4'
)
mapbox_map.show()

## 결론

1. 모든 유형의 사고 발생 건수는 차량등, 보행등과 양의 상관관계가 있다.
  - 그 중 차대차의 상관관계는 0.5이상으로 높은 편이며, 차대 사람도 낮지 않다.
2. 모든 유형의 사고 발생 건수는 안전지대의 개수와 양의 상관관계가 있다.
  - 신호등보다는 상관관계가 떨어지지만 차대차 0.5, 차대사람 0.33으로 적지않은 관계가 있는것으로 보인다

--------------------
# 전처리를 위한 EDA
--------------------

---------------

In [ ]:
# 실행

In [ ]:
pip install matplotlib==2.2.3

In [ ]:
# 커널 다시 시작

----------

In [ ]:
from geoband.API import *

import math
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
import geopy.distance

import pydeck as pdk
import shapely
import shapefile
import os
import re

from plotnine import *
import mapboxgl
from mapboxgl.viz import *
import seaborn as sns

import platform
import requests; from urllib.parse import urlparse

import warnings
warnings.filterwarnings(action='ignore') #경고 메시지 무시
from IPython.display import display #print가 아닌 display()로 연속 출력

In [ ]:
daejeon = gpd.read_file('daejeon_data.geojson')

In [ ]:
daejeon = daejeon[daejeon.isroad==1]

## 데이터 분할 : car_to_car / car_to_human / car_alone

In [ ]:
var_list = ['gid', 'geometry',
            
            'acci_cnt', 'acci_risk', 'acci_ratio',
            'car_car', 'acci_risk_ctc', 'acci_ratio_ctc',
            'car_human', 'acci_risk_cth', 'acci_ratio_cth',
            'car_only', 'acci_risk_cal', 'acci_ratio_cal',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt', 
            
            'youth_attack', 'senior_attack', 'weak_attack', 'attack_na',
            'youth_victim', 'senior_victim', 'weak_victim', 'victim_na', 'isroad',
            ]

In [ ]:
car_to_car = daejeon.copy()

var_list = ['gid', 'geometry',
            
#            'acci_cnt', 'acci_risk', 'acci_ratio',
            'car_car', 'acci_risk_ctc', 'acci_ratio_ctc',
#            'car_human', 'acci_risk_cth', 'acci_ratio_cth',
#            'car_only', 'acci_risk_cal', 'acci_ratio_cal',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt', 
            
#            'youth_attack', 'senior_attack', 'weak_attack', 'attack_na',
#            'youth_victim', 'senior_victim', 'weak_victim', 'victim_na', 'isroad',
            ]

car_to_car = car_to_car.loc[:, var_list]

In [ ]:
car_to_human = daejeon.copy()

var_list = ['gid', 'geometry',
            
#            'acci_cnt', 'acci_risk', 'acci_ratio',
#            'car_car', 'acci_risk_ctc', 'acci_ratio_ctc',
            'car_human', 'acci_risk_cth', 'acci_ratio_cth',
#            'car_only', 'acci_risk_cal', 'acci_ratio_cal',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt', 
            
#            'youth_attack', 'senior_attack', 'weak_attack', 'attack_na',
#            'youth_victim', 'senior_victim', 'weak_victim', 'victim_na', 'isroad',
            ]

car_to_human = car_to_human.loc[:, var_list]

In [ ]:
car_alone = daejeon.copy()

var_list = ['gid', 'geometry',
            
#            'acci_cnt', 'acci_risk', 'acci_ratio',
#            'car_car', 'acci_risk_ctc', 'acci_ratio_ctc',
#            'car_human', 'acci_risk_cth', 'acci_ratio_cth',
            'car_only', 'acci_risk_cal', 'acci_ratio_cal',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt', 
            
#            'youth_attack', 'senior_attack', 'weak_attack', 'attack_na',
#            'youth_victim', 'senior_victim', 'weak_victim', 'victim_na', 'isroad',
            ]

car_alone = car_alone.loc[:, var_list]

## 데이터에 DUMMY VARIABLE 추가

In [ ]:
def make_binary(data, var_list):
    
    df = data.copy()
    
    for var_name in var_list : 
        new_var_name = var_name + '_bin'
        df[new_var_name] = [1 if x>0 else 0 for x in df[var_name]]
        
    return df

In [ ]:
var_list = [            
            'car_car', 'acci_risk_ctc', 'acci_ratio_ctc',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt'
            ]

car_to_car_bin = make_binary(data=car_to_car, var_list=var_list)

In [ ]:
var_list = [            
            'car_human', 'acci_risk_cth', 'acci_ratio_cth',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt'
            ]

car_to_human_bin = make_binary(data=car_to_human, var_list=var_list)

In [ ]:
var_list = [            
            'car_only', 'acci_risk_cal', 'acci_ratio_cal',
                        
            'traffic_total_sum_inter', 'freq_max_inter', 'time_max_inter',
            'speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110',
            'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 
            'express', 'general', 'oneway', 'car_lane', 'intersects', 'TURN_P', 
            'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
            'sign_traffic', 'safe_zone', 'barrier_road',
            
            'pop_all', 'pop_sen', 'pop_prd', 'pop_chd', 'cars_cnt', 
            'building_tot', 'house', 'office', 'other',
            'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt', 'restaurant_cnt'
            ]

car_alone_bin = make_binary(data=car_alone, var_list=var_list)

## 교통량 변수1 : traffic

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='traffic_total_sum_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='traffic_total_sum_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + stat_smooth(method='lm', color='red')
)

#### 차대사람 사고의 경우 범주화된 traffic 변수와의 관계 고려

In [ ]:
(
    ggplot(car_to_human, aes(x='traffic_total_sum_inter'))
    + geom_density()
)

In [ ]:
# traffic 변수 분포의 quantile을 이용해 범주화

qmin = min(car_to_human.traffic_total_sum_inter)
q1 = car_to_human.traffic_total_sum_inter.quantile(q=0.25, interpolation='nearest')
q2 = car_to_human.traffic_total_sum_inter.quantile(q=0.5, interpolation='nearest')
q3 = car_to_human.traffic_total_sum_inter.quantile(q=0.75, interpolation='nearest')
qmax = max(car_to_human.traffic_total_sum_inter)

qmin, q1, q2, q3, qmax

In [ ]:
car_to_human_bin['traffic_total_sum_inter_categ'] = [1 if x<q1 else (2 if x<q2 else (3 if x<q3 else 4)) for x in car_to_human_bin['traffic_total_sum_inter']]


(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='traffic_total_sum_inter_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
car_to_human_bin.traffic_total_sum_inter_categ = car_to_human_bin.traffic_total_sum_inter_categ.astype('category')

(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='traffic_total_sum_inter_categ'))
    + geom_violin()
)

<span style='color:Red'> 차대사람 사고의 경우 범주화된 traffic 변수 이용이 유의미할 수 있음

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='traffic_total_sum_inter'))
    + geom_jitter(size=0.1, alpha=0.3, width = 0.05, height = 0.3) + geom_smooth(color='red')
)

In [ ]:
car_alone_bin['traffic_total_sum_inter_categ'] = car_to_human_bin['traffic_total_sum_inter_categ'].astype('category')

(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='traffic_total_sum_inter_categ'))
    + geom_violin()
)

## 교통량 변수2 : freq

In [ ]:
(
    ggplot(car_to_car, aes(x='freq_max_inter'))
    + geom_histogram()
)

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='freq_max_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='freq_max_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='freq_max_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

## 교통량 변수3 : time

In [ ]:
(
    ggplot(car_to_car, aes(x='time_max_inter'))
    + geom_histogram()
)

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='time_max_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='time_max_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='time_max_inter'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

## 교통량 변수 2~3 : freq vs time

In [ ]:
(
    ggplot(car_to_car, aes(y='time_max_inter', x='freq_max_inter'))
    + geom_point(size=0.1, alpha=0.3)
)

<span style='color:Red'> time 변수와 freq 변수의 상관성이 크게 보임. 분포가 고른 freq 변수를 대신 사용

## 교통량 변수 4 : speed

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_0'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_30'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_40_to_50'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_60_to_70'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_80_to_110'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> speed 변수의 의미 : 격자에 해당 속도의 도로가 몇개 지나가는지? 이는 분석에 바로 이용되기에 무리가 있음. 따라서 speed 변수를 이용해서 분석에 유의미한 영향을 미칠 수 있도록 파생변수 도출이 필요

In [ ]:
# 격자별로 지나는 도로 중 최빈 도로속도 범주화

def mainstream(vec):
    if max(vec)==vec[0]:
        return 'categ1'
    elif max(vec)==vec[1]:
        return 'categ2'
    elif max(vec)==vec[2]:
        return 'categ3'
    elif max(vec)==vec[3]:
        return 'categ4'
    else:
        return 'categ5'
    
temp = car_to_car_bin.loc[:, ['speed_0', 'speed_30', 'speed_40_to_50', 'speed_60_to_70', 'speed_80_to_110']]
car_to_car_bin['speed_main_categ'] = temp.apply(mainstream, axis=1)

In [ ]:
# 격자별로 지나는 도로 중 가장 속도가 높은 도로의 범주화

def get_max(vec):
    idx = np.dot(np.transpose(vec), np.diag(range(1, len(vec)+1)))
    return max(idx)

temp = car_to_car_bin.loc[:, ['speed_0_bin', 'speed_30_bin', 'speed_40_to_50_bin', 'speed_60_to_70_bin', 'speed_80_to_110_bin']]
car_to_car_bin['speed_max_categ'] = temp.apply(get_max, axis=1)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_main_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='speed_max_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
car_to_car_bin['speed_max_category'] = car_to_car_bin.speed_max_categ.astype('category')

(
    ggplot(car_to_car_bin, aes(x='acci_risk_ctc', color='speed_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 10]) + scale_y_continuous(limits=[0, 0.5])
)

In [ ]:
car_to_car_bin['speed_max_category'] = car_to_car_bin.speed_max_categ.astype('category')

(
    ggplot(car_to_car_bin, aes(x='acci_risk_ctc', color='speed_max_category'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 10]) + scale_y_continuous(limits=[0, 0.5])
)

<span style='color:Red'> 확실히 3,4 범주에 비해서 1,2,5 범주의 격자에서 사고가 더 많이 관측됨을 알 수 있음

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_0'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_30'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_40_to_50'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_60_to_70'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_80_to_110'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
# 격자별로 지나는 도로 중 최빈 도로속도 범주
# 격자별로 지나는 도로 중 가장 속도가 높은 도로의 범주
    
car_to_human_bin['speed_main_categ'] = car_to_car_bin['speed_main_categ']
car_to_human_bin['speed_max_categ'] = car_to_car_bin['speed_max_categ']
car_to_human_bin['speed_max_category'] = car_to_car_bin['speed_max_categ'].astype('category')

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_main_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='speed_max_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(x='acci_risk_cth', color='speed_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 10]) + scale_y_continuous(limits=[0, 0.5])
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(x='acci_risk_cth', color='speed_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 5]) + scale_y_continuous(limits=[0, 0.5])
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(x='acci_risk_cth', color='speed_max_category'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 10]) + scale_y_continuous(limits=[0, 0.5])
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(x='acci_risk_cth', color='speed_max_category'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 5]) + scale_y_continuous(limits=[0, 0.5])
)

<span style='color:Red'> 차대사람의 경우 3,4 범주와 1,2 범주가 비슷한 양상을 보이지만 이와 달리 5 범주에서는 조금 다름

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_0'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_30'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_40_to_50'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_60_to_70'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_80_to_110'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
# 격자별로 지나는 도로 중 최빈 도로속도 범주
# 격자별로 지나는 도로 중 가장 속도가 높은 도로의 범주
    
car_alone_bin['speed_main_categ'] = car_to_car_bin['speed_main_categ']
car_alone_bin['speed_max_categ'] = car_to_car_bin['speed_max_categ']
car_alone_bin['speed_max_category'] = car_to_car_bin['speed_max_categ'].astype('category')

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_main_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='speed_max_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(x='acci_risk_cal', color='speed_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 5]) + scale_y_continuous(limits=[0, 1])
)

In [ ]:
car_to_car_bin['speed_max_category'] = car_to_car_bin.speed_max_categ.astype('category')

(
    ggplot(car_alone_bin, aes(x='acci_risk_cal', color='speed_max_category'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 5]) + scale_y_continuous(limits=[0, 1])
)

<span style='color:Red'> 차량단독의 경우 전반적으로 모든 범주가 비슷함

## 교통량 변수 5 : width

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_1'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_2'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_3'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_4'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_5'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> width 변수 또한 분석에 유의미한 영향을 미칠 수 있도록 파생변수 도출이 필요

In [ ]:
# 격자별로 지나는 도로 중 최빈 도로 차선 범주

def mainstream(vec):
    if max(vec)==vec[0]:
        return 'categ1'
    elif max(vec)==vec[1]:
        return 'categ2'
    elif max(vec)==vec[2]:
        return 'categ3'
    elif max(vec)==vec[3]:
        return 'categ4'
    else:
        return 'categ5'
    
temp = car_to_car_bin.loc[:, ['width_1', 'width_2', 'width_3', 'width_4', 'width_5']]
car_to_car_bin['width_main_categ'] = temp.apply(mainstream, axis=1)

In [ ]:
# 격자별로 지나는 도로 중 가장 넓은 차선의 도로의 범주

def get_max(vec):
    idx = np.dot(np.transpose(vec), np.diag(range(1, len(vec)+1)))
    return max(idx)

temp = car_to_car_bin.loc[:, ['width_1_bin', 'width_2_bin', 'width_3_bin', 'width_4_bin', 'width_5_bin']]
car_to_car_bin['width_max_categ'] = temp.apply(get_max, axis=1)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_main_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='width_max_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(x='acci_risk_ctc', color='width_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 15]) + scale_y_continuous(limits=[0, 0.5])
)

In [ ]:
car_to_car_bin['width_max_categ'] = car_to_car_bin.width_max_categ.astype('category')

(
    ggplot(car_to_car_bin, aes(x='acci_risk_ctc', color='width_max_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 15]) + scale_y_continuous(limits=[0, 0.5])
)

<span style='color:Red'> 차대차의 경우 4,5 범주가 비슷한 양상을 보이고 그 외가 비슷해보임

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_1'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_2'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_3'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_4'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_5'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
# 격자별로 지나는 도로 중 최빈 도로 차선 범주
# 격자별로 지나는 도로 중 가장 넓은 차선의 도로의 범주

car_to_human_bin['width_main_categ'] = car_to_car_bin['width_main_categ']
car_to_human_bin['width_max_categ'] = car_to_car_bin['width_max_categ']
car_to_human_bin['width_max_category'] = car_to_car_bin['width_max_categ'].astype('category')

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_main_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='width_max_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(x='acci_risk_cth', color='width_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 15]) + scale_y_continuous(limits=[0, 0.5])
)

In [ ]:
car_to_car_bin['width_max_categ'] = car_to_car_bin.width_max_categ.astype('category')

(
    ggplot(car_to_human_bin, aes(x='acci_risk_cth', color='width_max_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 15]) + scale_y_continuous(limits=[0, 0.5])
)

<span style='color:Red'> 차대사람의 경우 4,5 범주가 비슷한 양상을 보이고 그 외가 비슷함

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_1'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_2'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_3'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_4'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_5'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
# 격자별로 지나는 도로 중 최빈 도로 차선 범주
# 격자별로 지나는 도로 중 가장 넓은 차선의 도로의 범주

car_alone_bin['width_main_categ'] = car_to_car_bin['width_main_categ']
car_alone_bin['width_max_categ'] = car_to_car_bin['width_max_categ']
car_alone_bin['width_max_category'] = car_to_car_bin['width_max_categ'].astype('category')

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_main_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='width_max_categ'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(x='acci_risk_cal', color='width_main_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 5]) + scale_y_continuous(limits=[0, 1])
)

In [ ]:
car_to_car_bin['width_max_categ'] = car_to_car_bin.width_max_categ.astype('category')

(
    ggplot(car_alone_bin, aes(x='acci_risk_cal', color='width_max_categ'))
    + geom_density(alpha=0.5) 
    + scale_x_continuous(limits=[0, 5]) + scale_y_continuous(limits=[0, 1])
)

<span style='color:Red'> 차량단독의 경우 범주별로 큰 차이가 보이지 않음

## 교통량 변수 6 : general, express

In [ ]:
(
    ggplot(car_alone_bin, aes(y='express_bin', x='general_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> express or general 등의 dummy 변수로 바꿀 필요 있음

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='express_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='general_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차대차 사고는 express에서는 적게, general에서는 더 많이 있음

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='express_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='general_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차대사람 사고는 express에서는 현저히 적게, general에서는 더 많이 있음. 특히 차대차와 비교하면 차대사람 사고는 express인 격자에서 발생하는 빈도가 더 작음

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='express_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='general_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차량단독 사고는 고속도로에서는 더 적게, 일반도로에서는 더 많이 발생

## 교통량 변수 8 : car_lane

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='car_lane_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차대차 사고는 자동차전용도로에서 적게 발생

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='car_lane_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차대사람 사고는 자동차 전용도로에서 더 적게 발생

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='car_lane_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차량단독 사고는 자동차전용도로에서 더 적게 발생하기는 하지만 차대차, 차대사람 사고와 비교할 경우에는 자동차전용도로에서 사고가 조금 더 발생함

## 교통량 변수 9 : intersects

### 차대차

In [ ]:
(
    ggplot(car_to_car_bin, aes(y='acci_risk_ctc', x='intersects_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차대차 사고는 교차로가 있는 곳에서 더 발생

### 차대사람

In [ ]:
(
    ggplot(car_to_human_bin, aes(y='acci_risk_cth', x='intersects_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

### 차량단독

In [ ]:
(
    ggplot(car_alone_bin, aes(y='acci_risk_cal', x='intersects_bin'))
    + geom_jitter(size=0.1, alpha=0.3) + geom_smooth(color='red')
)

<span style='color:Red'> 차량단독 사고는 교차로의 영향이 크다고 보기 힘들 것